In [1]:
!pip install pyarrow 
!pip install sqlparse
!pip uninstall --yes autoai-libs tensorflow-text numba numpy
!pip install numpy==1.22.4
!pip install --upgrade ibmcloudsql


Found existing installation: numpy 1.23.1
Uninstalling numpy-1.23.1:
  Successfully uninstalled numpy-1.23.1
  Using cached numpy-1.22.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)


In [2]:
import ibmcloudsql
import sqlparse 
from pygments import highlight
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter, Terminal256Formatter
lexer = get_lexer_by_name("sql", stripall=True)
formatter = Terminal256Formatter(style='vim')

In [3]:
apikey='VzOR-c4M0WIDDJVRmAHCRgkjidap41sYcrMhrJTkUdtH'
instancecrn='crn:v1:bluemix:public:sql-query:us-south:a/713c783d9a507a53135fe6793c37cc74:3d9a7cc0-f590-4293-9849-1960d537e1a9::'
dataengineurl='cos://s3.us-south.cloud-object-storage.appdomain.cloud/dataengine002/notebook'
logsurl="cos://s3.us-south.cloud-object-storage.appdomain.cloud/activitytrackerarchiving002"


In [4]:
apikey='aGosHjLCG98eeoiGwZzpbFwMJB4LnBQBEbmFHPuPAwfe'
instancecrn='crn:v1:bluemix:public:sql-query:us-south:a/713c783d9a507a53135fe6793c37cc74:436fd1b2-937b-4d4c-8302-0ae214a5003a::'
dataengineurl='cos://s3.us-south.cloud-object-storage.appdomain.cloud/logarchive-data-engine-001'
logsurl='cos://s3.us-south.cloud-object-storage.appdomain.cloud/logarchive-001'

In [5]:
# fixup testing todo
apikey='VzOR-c4M0WIDDJVRmAHCRgkjidap41sYcrMhrJTkUdtH'


In [6]:
sqlClient = ibmcloudsql.SQLQuery(apikey, instancecrn, client_info='notebook', target_cos_url=dataengineurl, max_concurrent_jobs=4, max_tries=3 )
#sqlClient.configure()  # use this if you want to change the API key or Data Engine CRN later
    
sqlClient.logon()
print('\nYour Data Engine web console link:\n')
sqlClient.sql_ui_link()


Your Data Engine web console link:

https://dataengine.cloud.ibm.com/sqlquery/?instance_crn=crn:v1:bluemix:public:sql-query:us-south:a/713c783d9a507a53135fe6793c37cc74:436fd1b2-937b-4d4c-8302-0ae214a5003a::


'https://dataengine.cloud.ibm.com/sqlquery/?instance_crn=crn:v1:bluemix:public:sql-query:us-south:a/713c783d9a507a53135fe6793c37cc74:436fd1b2-937b-4d4c-8302-0ae214a5003a::'

In [7]:
def sql_format(sql):
    formatted_sql = sqlparse.format(sql, reindent=True, indent_tabs=True, keyword_case='upper')
    return highlight(formatted_sql, lexer, formatter)

def sql_format_print(sql):
    print('\nYour SQL statement is:\n')
    print(sql_format(sql))

def sql_into(sql, intourl):
    if " INTO " not in sql:
        return sql + f' INTO {intourl} STORED AS JSON'
    return sql

def sql_r(sql):
    global dataengineurl
    sql=sql_into(sql, dataengineurl)
    sql_format_print(sql)
    result_df = sqlClient.run_sql(sql)
    print(result_df.head(10))
    return result_df



In [8]:
result_df = sql_r(f'SELECT year, month, _source__line FROM FLATTEN({logsurl} STORED AS JSON) WHERE _source_outcome NOT REGEXP "success" LIMIT 10')
result_df.head(100)


Your SQL statement is:

SELECT YEAR,
	MONTH,
	_source__line
FROM FLATTEN(cos://s3.us-south.cloud-OBJECT-storage.appdomain.cloud/logarchive-001 STORED AS JSON)
WHERE _source_outcome NOT REGEXP "success"
LIMIT 10 INTO cos://s3.us-south.cloud-OBJECT-storage.appdomain.cloud/logarchive-DATA-ENGINE-001 STORED AS JSON



ValueError: Protocol not known: image:r006-774b8de7-3d9a-4692-bb36-157c710b804a\",\"typeURI\":\"is.image/image\",\"name\":\"usc4-travis-72589103\",\"host\":{\"address\":\"https

In [ ]:
result_df["_source__line"].head(2)

In [9]:
!pip freeze

absl-py==0.12.0
aiohttp==3.8.1
aiosignal==1.2.0
appdirs==1.4.4
argon2-cffi==20.1.0
astropy==5.0
astunparse==1.6.3
async-generator==1.10
async-timeout==4.0.1
attrs==21.2.0
autoai-ts-libs==1.1.9
autovizwidget==0.18.0
av==8.0.3
azure-core==1.21.0
azure-cosmos==4.2.0
backcall==0.2.0
backoff==1.10.0
beautifulsoup4==4.10.0
biopython==1.78
black==19.10b0
bleach==4.0.0
blinker==1.4
bokeh==2.3.3
boto3==1.18.21
botocore==1.21.41
Bottleneck==1.3.2
brotlipy==0.7.0
ca-data-connector @ file:///tmp/ws-build/install/ca-data-connector-python-11.2.0.tgz
cached-property==1.5.2
cachetools==4.2.2
cassandra-driver==3.25.0
certifi==2022.6.15
cffi==1.14.6
cfgv==3.3.1
charset-normalizer==2.0.4
click==8.0.4
cloudpickle==1.6.0
cplex==20.1.0.2
cryptography==3.4.8
cx-Oracle==8.3.0
cycler==0.11.0
cytoolz==0.11.0
dask==2021.10.0
debugpy==1.5.1
decision-optimization-client==1.0.0
decorator==5.1.0
defusedxml==0.7.1
Deprecated==1.2.13
dill==0.3.4
distlib==0.3.5
dm-tree==0.1.5
docplex==2.22.213
entrypoints==0.3
et-xmlfi